In [ ]:
!pip install requests
!pip install beautifulsoup4


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
import os
# Site web à scraper
url = 'https://www.monster.com/jobs/q-work-from-home-jobs?page=10&so=p.h.p'

# Faire la requête HTTP avec requests
response = requests.get(url)
# Afficher le code de statut de la requête
print(response.status_code)


200


In [ ]:
# Afficher les en-têtes de la réponse
print(response.headers)

# Afficher le contenu HTML de la page
html_content = response.text
print(html_content)



{'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Server': 'istio-envoy', 'Date': 'Fri, 24 Nov 2023 10:49:33 GMT', 'access-control-expose-headers': 'Server-Timing', 'server-timing': 'traceparent;desc="00-2411db25a60f5ec421eac787fe93faee-8cab3ec638cca568-01"', 'x-xss-protection': '1; mode=block', 'Set-Cookie': 'AMP_4a52be5521=eyJkZXZpY2VJZCI6IjYwNzBkNWExLThhZGUtNGIzOS05ZTY5LThkNzJlNGUyY2U3YiJ9; Domain=.monster.com; Path=/; Secure, mn_ja3=a48c0d5f95b1ef98f560f324fd275da1;Path=/;Max-Age=31536000', 'Cache-Control': 'private, no-cache, no-store, max-age=0, must-revalidate', 'Content-Encoding': 'gzip', 'x-envoy-upstream-service-time': '114', 'x-frame-options': 'SAMEORIGIN', 'strict-transport-security': 'max-age=31536000; includeSubDomains; preload', 'x-content-type-options': 'nosniff', 'referrer-policy': 'strict-origin', 'x-ja3-fingerprint': 'a48c0d5f95b1ef98f560f324fd275da1', 'Vary': 'Accept-Encoding', 'X-Cache': 'Miss from cloudfront'

In [ ]:
# Parser le contenu HTML avec BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Récupérer le titre de la page
title = soup.find('title')
print(title.text if title else "No title found")




Work From Home Jobs | Monster.com


In [ ]:
# Récupérer tous les liens de la page
links = soup.find_all('a', href=True)
for link in links:
    print(link['href'])



https://www.monster.com
https://www.monster.com
https://www.monster.com/salary/
https://www.monster.com/career-advice
https://www.monster.com/resumes/writing-services
https://www.monster.com/upload-resume/
https://hiring.monster.com/?intcid=skr_www_TopNav_Employer&ch=MONS
https://www.monster.com/profile/detail
https://www.monster.com/message-center
https://www.monster.com/profile/saved-jobs
/jobs
https://www.monster.com/job-openings/service-center-representative-remote-bethesda-md--20b4f6b7-0fa8-47e3-9399-6591aed41b57?mstr_dist=true
https://www.monster.com/job-openings/ar-ar-specialist-x-3-1-day-remote-thursday-only-alpharetta-ga--75823a71-519f-4b16-8d72-c936c9d82b0c?mstr_dist=true
https://www.monster.com/job-openings/gcs-lead-consultant-alpharetta-ga--822cbb5e-69f3-4c78-910b-3c7dca47c5b4?mstr_dist=true
https://www.monster.com/job-openings/junior-transmission-line-engineer-charlotte-nc--601684ee-6ce1-459a-98dd-ee841c059b3a?mstr_dist=true
https://www.monster.com/job-openings/outside-sal

In [ ]:
import requests
from bs4 import BeautifulSoup
import urllib.parse

# Votre code pour récupérer les liens et la soupe (soup) ici...

base_url = 'https://www.monster.com/jobs/q-work-from-home-jobs?page={page_num}&so=p.h.p'  # L'URL de base pour les liens relatifs

for link in links[:3]:  # Exemple: Seulement les deux premiers liens
    new_url = link['href']
    if not new_url.startswith('http'):
        new_url = urllib.parse.urljoin(base_url, new_url)
    try:
        new_rep = requests.get(new_url)
        new_soup = BeautifulSoup(new_rep.text, 'html.parser')
        new_title = new_soup.find('title')
        if new_title:
            print(f"Titre de la page {new_url}:")
            print(new_title.text)
            print("\n")
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération de {new_url}: {e}")


Titre de la page https://www.monster.com:
Monster Jobs - Job Search, Career Advice & Hiring Resources | Monster.com


Titre de la page https://www.monster.com:
Monster Jobs - Job Search, Career Advice & Hiring Resources | Monster.com


Titre de la page https://www.monster.com/salary/:
Salary Comparison and Salary Estimator | Monster.com




In [ ]:
import requests
from bs4 import BeautifulSoup

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

try:
    for page_num in range(1, 201):  # Scraper les pages de 1 à 41
        url = f'https://www.monster.com/jobs/q-work-from-home-jobs?page={page_num}&so=p.h.p'

        # Retrieve the page content
        response = requests.get(url, headers=headers)

        # Check the status code
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')

            # Find all job listings
            job_listings = soup.find_all('div', class_='cached-bot-shared-skeletonstyle__CachedSkeletonCardHeader-sc-15ar3p6-2 kVnHvr')

            # Check if job listings are available
            if job_listings:
                for job in job_listings:
                    job_title_elem = job.find('div', class_='cached-bot-shared-skeletonstyle__CachedSkeletonCardHeaderTitle-sc-15ar3p6-3')
                    company_elem = job.find('h2', class_='cached-bot-sharedstyle__JobCardCompany-sc-13tnr74-1')
                    location_elem = job.find_all('p', class_='cached-bot-sharedstyle__JobCardDetails-sc-13tnr74-3 fPLwxY')[-1]

                    # Extract job details if elements are found
                    job_title = job_title_elem.text.strip() if job_title_elem else 'Not found'
                    company = company_elem.text.strip() if company_elem else 'Not found'
                    location = location_elem.text.strip() if location_elem else 'Not found'

                    # Format and print the output for each job
                    output = f"Page {page_num}\nJob Title: {job_title}\nCompany: {company}\nLocation: {location}\n"
                    print(output)
            else:
                print(f"No job listings found on page {page_num}")

        else:
            print(f"Failed to retrieve the webpage for page {page_num}")

except Exception as e:
    print(f"An error occurred: {str(e)}")




Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Page 89
Job Title: Firmware Engineer - REMOTE
Company: Actalent
Location: Pasadena, CA

Page 90
Job Title: Service Center Representative - Remote
Company: Nesco Resource, LLC
Location: Bethesda, MD

Page 90
Job Title: AR- AR Specialist x 3- 1 day remote (Thursday only)
Company: Vaco, LLC
Location: Alpharetta, GA

Page 90
Job Title: Outside Sales Representative
Company: Inceed
Location: Remote, undefined

Page 90
Job Title: SQL Developer (Fully Remote)
Company: Vaco, LLC
Location: Brentwood, TN

Page 90
Job Title: Principal Associate, Data Loss Prevention (Remote-Eligible)
Company: Capital One
Location: Fredericksburg, VA

Page 90
Job Title: Systems Analyst - HRIS (Remote)
Company: Sentara
Location: Norfolk, VA

Page 90
Job Title: Remediation Data Analyst- SAS/SQL, 100% Remote W2 Only
Company: TEKsystems
Location: Chicago, IL

Page 90
Job Title: *Fully Remote* Structural Engineer
Company: Actalent
Location: Ne

In [ ]:
#de meme, mais ici on transforme resultats en EXCEL


import requests
from bs4 import BeautifulSoup
import pandas as pd

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

job_details = []  # Liste pour stocker les détails des offres d'emploi

try:
    for page_num in range(1, 202):  # Scraper les pages de 1 à 201
        url = f'https://www.monster.com/jobs/q-work-from-home-jobs?page={page_num}&so=p.h.p'

        # Retrieve the page content
        response = requests.get(url, headers=headers)

        # Check the status code
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')

            # Find all job listings
            job_listings = soup.find_all('div', class_='cached-bot-shared-skeletonstyle__CachedSkeletonCardHeader-sc-15ar3p6-2 kVnHvr')

            # Check if job listings are available
            if job_listings:
                for job in job_listings:
                    job_title_elem = job.find('div', class_='cached-bot-shared-skeletonstyle__CachedSkeletonCardHeaderTitle-sc-15ar3p6-3')
                    company_elem = job.find('h2', class_='cached-bot-sharedstyle__JobCardCompany-sc-13tnr74-1')
                    location_elem = job.find_all('p', class_='cached-bot-sharedstyle__JobCardDetails-sc-13tnr74-3 fPLwxY')[-1]

                    # Extract job details if elements are found
                    job_title = job_title_elem.text.strip() if job_title_elem else 'Not found'
                    company = company_elem.text.strip() if company_elem else 'Not found'
                    location = location_elem.text.strip() if location_elem else 'Not found'

                    # Ajouter les détails dans la liste
                    job_details.append({
                        'Job Title': job_title,
                        'Company': company,
                        'Location': location
                    })

            else:
                print(f"No job listings found on page {page_num}")

        else:
            print(f"Failed to retrieve the webpage for page {page_num}")

except Exception as e:
    print(f"An error occurred: {str(e)}")

# Convertir la liste de détails en DataFrame
df = pd.DataFrame(job_details)

# Enregistrer le DataFrame dans un fichier Excel
df.to_excel('job_listings3.xlsx', index=False)
